In [ ]:
import sys
sys.path.append("../")
from config import *

In [ ]:
!pip install -r "{PROJECT_ROOT}/requirements.txt"

In [ ]:
sys.path.append(f'{PROJECT_ROOT}')
sys.path.append(f'{PROJECT_ROOT}/deps/BARTScore')
sys.path.append(f'{PROJECT_ROOT}/deps/bert_score')
sys.path.append(f'{PROJECT_ROOT}/RL_bart_xsum/trl/trl')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, BartTokenizer, BartModel, BartForConditionalGeneration, BartConfig, GPT2Config,GPT2LMHeadModel
from transformers import top_k_top_p_filtering, GPT2Model
from transformers import GPT2Tokenizer, AutoModel, BartTokenizer, AutoTokenizer
from transformers import BertModel, BertPreTrainedModel
import torch
from torch import nn
from torch.nn import Identity
import torch.nn.functional as F
import sys
from bart_xsum import ValueHead
import ppo

import numpy as np
import os
import pandas as pd
from threading import active_count
import sys
from bart_score import BARTScorer
from bert_score import score
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize


In [ ]:
# pull data only pulls changed data
!dvc pull -f

## Code Section

In [ ]:
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deteministic = True
setup_seed(57)

In [ ]:
class BertRegresser(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        #The output layer that takes the [CLS] representation and gives an output
        self.regressor = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(768, 1))

    def forward(self, input_ids, attention_mask):
        #Feed the input to Bert model to obtain contextualized representations
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #Obtain the representations of [CLS] heads
        logits = outputs[1]
        output = self.regressor(logits)
        return output


config = BartConfig('facebook/bart-large-xsum', output_hidden_states=True)
model_name = f"{MODEL_DIR}/BART-XSum/"

print(DEVICE_ID)

#load preptrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, output_hidden_states=True).to(DEVICE_ID)
#load reference model
ref_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, output_hidden_states=True).to(DEVICE_ID)
#load reward model
reward_model = BertRegresser.from_pretrained('allenai/scibert_scivocab_uncased')

model_state, optimizer_state = torch.load(f"{PROJECT_ROOT}/reward_model/checkpoint") # <---------------------- !!!!!!!!!!!!!!!!!!!!!!!!!
reward_model.load_state_dict(model_state)
reward_model.to(DEVICE_ID)
#load Valuehead
hmodel = ValueHead(config).to(DEVICE_ID)

#load reward tokenizer
reward_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
#load model tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')

In [ ]:
# load annotated datad
anno_sample = pd.read_json(DATASET_140_ANNOTATED_JSON)
def creat_tupel(sample_np):
  ab = [sample_np[0]]*6
  ht = [sample_np[1]]*6
  indices = np.array([2, 5, 8, 11, 14, 17])
  sysms = list(sample_np[2].keys())
  sysms = ['original'] + sysms
  gents = list(sample_np[2].values())
  gents = [sample_np[1]] + gents
  gents_scores = list(sample_np[3].values())
  gents_scores = [gents_scores[i] for i in indices]
  max_idx = gents_scores.index(max(gents_scores))
  res = np.transpose(np.array([ab, ht, gents, gents_scores, sysms]))
  return res[max_idx,:].reshape(1,5), res
anno_sample_np =anno_sample.to_numpy()
res = []
res1 = []
for row in anno_sample_np:
  r1, r2 = creat_tupel(row)
  res += r1.tolist()
  res1 += r2.tolist()
gen_title_score_pairs = np.array(res1)
gen_title_score_pairs_bestone = np.array(res)

In [ ]:
"""# **3 Datasettings (only for analysis in default RL, final model using cross learning with all sample with best title)**"""

#@title pick bad annotated data or from bart-xsum
gen_title_score_pairs_bad = []
scores = gen_title_score_pairs_bestone[:,-2]
scores = np.array([float(s) for s in scores])
for row in gen_title_score_pairs_bestone:
  if float(row[-2]) <= np.mean(scores):
    gen_title_score_pairs_bad.append(row)

gen_title_score_pairs_bad = np.array(gen_title_score_pairs_bad)

#@title pick annotated data from bart-xsum
gen_title_score_pairs_xsum = []
for row in gen_title_score_pairs:
  if row[-1] == 'bart_xsum':
    gen_title_score_pairs_xsum.append(row)

gen_title_score_pairs_xsum = np.array(gen_title_score_pairs_xsum)

#@title pick good annotated data or from bart-xsum
gen_title_score_pairs_good = []
scores = gen_title_score_pairs_bestone[:,-2]
scores = np.array([float(s) for s in scores])
for row in gen_title_score_pairs_bestone:
  if float(row[-2]) >= np.mean(scores) or row[-1] == 'bart_xsum':
    gen_title_score_pairs_good.append(row)

gen_title_score_pairs_good = np.array(gen_title_score_pairs_good)

In [ ]:
"""# **2 Train Mode (Cross learning/ Default RL)**"""
gen_kwargs = {
    "min_length":-1,
    "top_k": 2,
    "top_p": 0.8,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    #"length_penalty" : -20.0,
    #"num_return_sequences" : 5,
    #"repetition_penalty" : 1.5,
}
def ACT_step(title_score_pairs, start_ids, stop_ids, tokenizer, model, ppo_trainer):
  res = []
  for i in range(start_ids, stop_ids):
    row = title_score_pairs[i]
    ref_response_txt = row[2]
    #model_name = row[4]
    original_title = row[1]
    query_txt = row[0]
    query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')
    #act step
    ref_response_tensor = tokenizer(ref_response_txt, return_tensors='pt').to('cuda')
     # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + ref_response_txt
    ref_reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    ref_reward = reward_model(ref_reward_encode['input_ids'], ref_reward_encode['attention_mask']).squeeze(-1)
    ref_reward = torch.tensor([ref_reward.item()]).to('cuda')
    #reward = torch.tensor(float(row[3])).to('cuda')
    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], ref_response_tensor["input_ids"], ref_reward)

    # get model response
    '''response_tensor = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                ).to('cuda')'''
    response_tensor = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    #test ref model response(generated title from fine-tunde bart-xsum)

    # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + response_txt
    reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    reward = reward_model(reward_encode['input_ids'], reward_encode['attention_mask']).squeeze(-1)
    reward = torch.tensor([reward.item()]).to('cuda')

    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], response_tensor, reward)

    #generate title with rewarded model
    '''new_model_response_ids = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                )'''
    new_model_response_ids = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    new_model_response_ids = new_model_response_ids.cpu().data.numpy()
    new_model_response_txt = tokenizer.batch_decode(new_model_response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    res.append([response_txt, new_model_response_txt])
    if i % 10 == 0:
      print('Original title: ', original_title)
      print('Generated reference title: ', ref_response_txt)
      print('RL-rewarded-after-step bart_xsum generated title: ', new_model_response_txt)
      print('- - - -'*20 + '>')
  return res

In [ ]:
gen_kwargs = {
    "min_length":-1,
    "top_k": 2,
    "top_p": 0.8,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    #"length_penalty" : 4.0,
    #"repetition_penalty" : 1.5,
}
def RL_steps(title_score_pairs, start_ids, stop_ids, tokenizer, model, ppo_trainer):
  res = []
  for i in range(start_ids, stop_ids):
    row = title_score_pairs[i]
    #model_name = row[4]
    original_title = row[1]
    query_txt = row[0]
    query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')
    # get model response
    '''response_tensor = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0,
                length_penalty=10.0,
                early_stopping = True,
                ).to('cuda')'''

    response_tensor = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    #test ref model response(generated title from fine-tunde bart-xsum)
    #ref_response_txt = row[2]

    # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + response_txt
    reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    reward = reward_model(reward_encode['input_ids'], reward_encode['attention_mask']).squeeze(-1)
    #reward = row[2].item()
    reward = torch.tensor([reward.item()]).to('cuda')

    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], response_tensor, reward)

    #generate title with rewarded model
    new_model_response_ids = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0,
                length_penalty=10.0,
                early_stopping = True,
                )
    new_model_response_ids = new_model_response_ids.cpu().data.numpy()
    new_model_response_txt = tokenizer.batch_decode(new_model_response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    res.append([response_txt, new_model_response_txt])
    if i % 6 == 0:
      print('Original title: ', original_title)
      #print('Generated reference title: ', ref_response_txt)
      print('RL-rewarded-after-step bart_xsum generated title: ', new_model_response_txt)
      print('- - - -'*20 + '>')
  return res

# test setup

In [ ]:

'''
df1 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_lowLR.csv', index_col=0)
df2 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_midLrEp.csv', index_col=0)
df3 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_verylowLR.csv', index_col=0)
df4 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_midLrEp_good_scores.csv', index_col=0)
ab = df1[['abstract']].to_numpy().squeeze()
ot = df1[['original title']].to_numpy().squeeze()
gt = df1[['generated title before RL']].to_numpy().squeeze()
t1 = df1[['generated title after RL']].to_numpy().squeeze()
t2 = df2[['title-xsum-reward']].to_numpy().squeeze()
t3 = df3[['generated title after RL']].to_numpy().squeeze()
t4 = df1[['generated title after RL']].to_numpy().squeeze()
ttt = np.array([ab,ot,gt,t1,t2,t3,t4]).transpose()
df = pd.DataFrame(ttt)
df.columns =['abstracz','human title', 'before RL', 'low LR', 'mid LR', 'very low LR', 'mid LR goog only']
df.to_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_dif_setup.csv')
'''


In [ ]:
"""# **Train Settings and Do Train**"""

#@title initialize PPOtrainer
# initialize trainer
#ppo_config = {'batch_size': 1, 'forward_batch_size': 1}

ppo_config = {
        "lr": 2e-6,#1.41e-5,#6e-7,#"lr": 3e-6,#,
        "adap_kl_ctrl": True,
        "init_kl_coef":0.2,
        "target": 6,
        "horizon":10000,
        "gamma":1,
        "lam":0.95,
        "cliprange": .2,
        "cliprange_value":.2,
        "vf_coef":.1,
        "batch_size": 1,
        "forward_batch_size": 1,
        "ppo_epochs": 3,
    }
ppo_trainer = PPOTrainer(model, ref_model, hmodel, **ppo_config)
res = ACT_step(gen_title_score_pairs_bestone, 0, len(gen_title_score_pairs_bestone), tokenizer, model, ppo_trainer)
#res = RL_steps(gen_title_score_pairs_bad, 0, len(gen_title_score_pairs_bad), tokenizer, model, ppo_trainer)

In [ ]:
query_txt = "This bachelor thesis explores the generation of title based on a given abstract using neural language model. Recently, neural language models have been used in many scenarios with practical applications. For example, in scientific writing, automatic summary generation from long texts is used to assist in the reading and selection of relevant scientific articles. Title is an important part of scientific article, but the title generation using neural language and optimization for neural language model based on human preferences are less studied. This thesis addresses this gap and presents an optimized model based on state-of-the-art pre-trained neural language model which generate human-preferred titles from a given abstract. The model is fine-tuned on datasets of scientific article and optimized from human preferences using the novel learning perspective in reinforcement learning environment. The result shows that, the neural language model have powerful capabilities on the abstract-to-title task and the reinforcement learning approach is effective in scalable learning of neural language model."

query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')

In [ ]:
ref_tensor=model.generate(input_ids = query_tensor["input_ids"], 
                          attention_mask = query_tensor["attention_mask"], 
                          max_new_tokens=30, 
                          **gen_kwargs)[-30:].to('cuda')

In [ ]:
ref_tensor = model.generate(
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0,
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')

In [ ]:
ref_txt = tokenizer.batch_decode(ref_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


# **Generation**



In [ ]:
outres = []
for row in gen_title_score_pairs_bestone:
  ab = row[0]
  ht = row[1]
  ogt = row[2]
  query_txt = ab
  query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')
  #get ref model response
  '''ref_tensor = ref_model.generate(
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0, 
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')'''
  ref_tensor=ref_model.generate(input_ids = query_tensor["input_ids"], 
                          attention_mask = query_tensor["attention_mask"], 
                          max_new_tokens=30, 
                          **gen_kwargs)[-30:].to('cuda')
  ref_txt = tokenizer.batch_decode(ref_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  # get model response
  '''response_tensor = model.generate(
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0, 
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')'''
  response_tensor = model.generate(input_ids = query_tensor["input_ids"], attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')

  response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  outres.append([ab, ht,ogt, ref_txt, response_txt])

In [ ]:
df = pd.DataFrame(outres)
df.columns = ['abstract', 'original title','best title' , 'generated title before RL', 'generated title after RL']
path=f'{OUTPUT_DIR}\act_'+ "{:.9f}".format(ppo_config['lr'])+'_'+str(ppo_config['ppo_epochs'])+'.csv'
df = df.drop(['original title'], axis=1)
df.columns = ['abstract', 'best title', 'title-xsum', 'title-xsum-reward']
df_np = df.to_numpy()

n_np = []
for row in df_np:
  ab = row[0]
  ts = row[1:]
  #ts[0] = ts[0]
  #ts[0] = ts[0]
  #ts = np.random.permutation(ts)
  n_np.append([ab, ts[1], ts[2]])
n_np = np.array(n_np)
se = pd.DataFrame(n_np[:30])

# **Analysis**

In [ ]:
# %cd /content/drive/MyDrive/Thesis/BARTScore

bart_scorer = BARTScorer(device='cuda:0',checkpoint='D:\\Thesis\\BARTScore-main\\bart_xsum')

# %cd /content/drive/MyDrive/Thesis/emnlp19-moverscore-master
'''sys.path.append('D:\\Thesis\\emnlp19-moverscore-master')
from moverscore_v2 import get_idf_dict, word_mover_score
from typing import List, Union, Iterable
from collections import defaultdict
import numpy as np
def sentence_score(hypothesis: str, references: List[str], trace=0):
    
    idf_dict_hyp = defaultdict(lambda: 1.)
    idf_dict_ref = defaultdict(lambda: 1.)
    
    hypothesis = [hypothesis] * len(references)
    
    sentence_score = 0 

    scores = word_mover_score(references, hypothesis, idf_dict_ref, idf_dict_hyp, stop_words=[], n_gram=1, remove_subwords=False)
    
    sentence_score = np.mean(scores)
    
    if trace > 0:
        print(hypothesis, references, sentence_score)
            
    return sentence_score
'''
# Commented out IPython magic to ensure Python compatibility.

In [ ]:
%cd /content/drive/MyDrive/Thesis/bert_score

In [ ]:
df = pd.read_csv(f'{OUTPUT_DIR}\output_111_annotated_from_bart_xsum_act_bad_final0.000002000_3.csv', index_col=0)[:30]

In [ ]:
abs = df['abstract'].to_list()
ots = df['original title'].to_list()
bts = df['best title'].to_list()
ogts = df['generated title before RL'].to_list()
rlts = df['generated title after RL'].to_list()

In [ ]:
scores11 = bart_scorer.score(abs, ots, batch_size=1)
scores21 = bart_scorer.score(abs, ogts, batch_size=1)
scores31 = bart_scorer.score(abs, rlts, batch_size=1)
'''
scores12 = [sentence_score(ot, [ab]) for ot,ab in zip(ots, abs)]
scores22 = [sentence_score(ogt, [ab]) for ogt,ab in zip(ogts, abs)]
scores32 = [sentence_score(rlt, [ab]) for rlt,ab in zip(rlts, abs)]'''

In [ ]:
p,r,score13 = score(ots, abs, lang="en")
p,r,score23 = score(ogts, abs, lang="en")
p,r,score33 = score(rlts, abs , lang="en")

In [ ]:
#BartScore
print('abs_ots: ', sum(scores11)/len(scores11))
print('abs_ogts: ', sum(scores21)/len(scores21))
print('abs_rlts: ', sum(scores31)/len(scores31))
#BertScore
print('abs_ots: ', np.array(score13.tolist()).mean())
print('abs_ogts: ', np.array(score23.tolist()).mean())
print('abs_rlts: ', np.array(score33.tolist()).mean())

In [ ]:
#MoverScore

print('abs_ots: ', sum(scores12)/len(scores12))
print('abs_ogts: ', sum(scores22)/len(scores22))
int('abs_rlts: ', sum(scores32)/len(scores32))

In [ ]:
r1 = [np.exp(-0.654991332689921), 0.8518536269664765, 0.5175741747308048]

In [ ]:
r2 = [np.exp(-0.6468217780192693), 0.8508123060067495, 0.5174440166876286]

In [ ]:
#r3 = [np.exp(-0.6468217780192693), 0.5174440166876286, 0.8508123060067495]

In [ ]:
normalized_metrics = normalize(np.array([r1, r2]), axis=0, norm='l1')
normalized_metrics

In [ ]:
normalized_metrics = normalize(np.array([r1, r2]), axis=0, norm='l1')
normalized_metrics